# Implemented Algorithms and Function

These functions: routines and algorithms are used throughout this notebook for computation and statistical plotting.

The computational / statistical algorithms and the parameters used are inspired from Poldrack, R., Mumford, J., & Nichols, T. (2011). *Handbook of Functional MRI Data Analysis*. Cambridge: Cambridge University Press. and  F., Gregory Ashby. (2019). *Statistical Analysis of fMRI Data*. MIT Press.



#### Imports

In [1]:
! pip install nilearn nistats

     |████████████████████████████████| 3.0MB 4.2MB/s 
     |████████████████████████████████| 122kB 48.6MB/s 


In [2]:
from warnings import filterwarnings
filterwarnings('ignore')
import numpy as np
import pandas as pd
from numpy import array
import matplotlib.pyplot as plt


import gzip, shutil
import time
import yaml
import glob

from nilearn import image, plotting
from nilearn.plotting import plot_stat_map
from nilearn.image import concat_imgs, mean_img
from nistats.design_matrix import make_first_level_design_matrix
from nistats.first_level_model import FirstLevelModel
from nistats.second_level_model import SecondLevelModel
from nilearn.plotting import plot_roi
from nilearn.masking import compute_background_mask
from nilearn.image import threshold_img



from nilearn.image import math_img
from nilearn import input_data
from nilearn.input_data import NiftiMasker

from nistats.reporting import plot_design_matrix,plot_contrast_matrix
from nistats.thresholding import map_threshold
from nistats.reporting import get_clusters_table
from nistats.experimental_paradigm import check_events

from sklearn.feature_selection import f_classif, mutual_info_classif, SelectFdr

from sklearn.feature_selection import GenericUnivariateSelect
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier        
    
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: 

 | Using Nistats with Nilearn versions >= 0.7.0 is redundant and potentially conflicting.
 | Nilearn versions 0.7.0 and up offer all the functionality of Nistats as well the latest features and fixes.
 | We strongly recommend uninstalling Nistats and using Nilearn's stats & reporting modules.



## GLM

In [3]:
def compute_core_fmri_glm(task,
                          data_path,
                          events_path,
                          motion_param_path,
                          cache_mem_path,
                          analysis_type,
                          param_modulation,
                          t_r,
                          slice_time_ref,
                          hrf_model,
                          drift_model,
                          drift_order,
                          fbr_expected_tr_delay,
                          pre_whitening_spec,
                          apply_trans_mask,
                          roi_img,
                          hpf_cut,
                          smoothing_fwhm,
                          standardize,
                          detrend,minimize_memory):
    
                          
                          
    # Data processing                      

    data = image.load_img(data_path)
    m_img = mean_img(data) # mean niimg for plotting purposes
    motion = pd.read_csv(motion_param_path,sep=',',header=None)
    motion = motion[list(range(1,7))]
    motion_col = ['tx', 'ty', 'tz', 'rx', 'ry', 'rz']
    motion.columns = motion_col



    #Experimental Paradigm Processing

    events = pd.read_csv(events_path,sep='\t')
    events.columns = ['onset','duration','trial_type','modulation']
    events.dropna(inplace=True)
    events.index = range(len(events))
    
    
    
    if analysis_type == 'High_Low_Pleasance':
        
        events['trial_type_desc'] = ['High Pleasance' if x >=50 else 'Low Pleasance' for x in events['modulation'] ]
        
        if param_modulation:
           
            events = events[['onset','duration','trial_type_desc','modulation']]
            events.columns = ['onset','duration','trial_type','modulation']
        else:
            events = events[['onset','duration','trial_type_desc']]
            events.columns = ['onset','duration','trial_type']
            


    elif analysis_type == 'High_Mid_Low_Pleasance':
        
        events['trial_type_desc'] = ['High Pleasance' if x >=67 else 'Mid Pleasance' if x>=34 else 'Low Pleasance' for x in events['modulation'] ]
        
        if param_modulation:
           
            events = events[['onset','duration','trial_type_desc','modulation']]
            events.columns = ['onset','duration','trial_type','modulation']
        else:
            events = events[['onset','duration','trial_type_desc']]
            events.columns = ['onset','duration','trial_type']
            
    
    # Design Matrix Computation + plotting
    
    frametimes = np.linspace(0, (data.shape[3] - 1) * t_r, data.shape[3])
    
    fir_delays = list(range(1,fbr_expected_tr_delay+1)) if fbr_expected_tr_delay != None else None
    
    design_matrix = make_first_level_design_matrix(frametimes,
                       events=events,
                       add_regs=motion,
                       add_reg_names=motion_col,
                       hrf_model=hrf_model,
                       drift_model= drift_model,
                       fir_delays= fir_delays, 
                       drift_order =drift_order)
    
    plot_design_matrix(design_matrix)
    
    # Statistically Enhancing Mask Computation
    
    if apply_trans_mask:
    
    
        masker = input_data.NiftiMasker(mask_img=roi_img,smoothing_fwhm=smoothing_fwhm,
                                standardize=standardize,detrend=detrend,
                                high_pass=hpf_cut,t_r=t_r).fit(data)
      
    else :
        
        masker = input_data.NiftiMasker(mask_img=roi_img).fit(data)
        
        
        
    # GLM instantiation and fit
    
    glm = FirstLevelModel(t_r=t_r,
                      slice_time_ref=slice_time_ref,
                      mask_img=masker,
                      noise_model= pre_whitening_spec,
                      high_pass=hpf_cut,
                      smoothing_fwhm = smoothing_fwhm,
                      standardize=standardize,
                      memory=cache_mem_path,
                      minimize_memory=minimize_memory, 
                      verbose=1)

    glm = glm.fit(data,design_matrices=design_matrix)
    
    return glm, design_matrix

## Statistical Plotting

In [4]:
def top_cluster_stat_results(data,task,glm,z_map,threshold,table,bg_img):
    
    # Top Cluster Selection + Masking
    coords = table.loc[range(1, 7), ['X', 'Y', 'Z']].values
    masker = input_data.NiftiSpheresMasker(coords,standardize=True)
    real_timeseries = masker.fit_transform(data)
    predicted_timeseries = masker.fit_transform(glm.predicted[0])

    
    


    # PLOT 1
    fig1, axs1 = plt.subplots(2, 3)
    for i in range(0, 3):
        axs1[0, i].set_title(' Top {} Activated Cluster \n(cluster peak voxel at coord {})'.format(i+1,coords[i]))
        axs1[0, i].plot(real_timeseries[:, i], c='blue', lw=2,label = 'Actual Bold Signal')
        axs1[0, i].plot(predicted_timeseries[:, i], c='r',  ls='-', lw=2,label= 'Predicted Bold Signal')
        axs1[0, i].set_xlabel('Time')
        axs1[0, i].set_ylabel('Signal intensity', labelpad=0)
        axs1[0, i].legend()
        roi_img = plotting.plot_stat_map(
            z_map, cut_coords=[coords[i][2]], threshold=threshold, figure=fig1,
            axes=axs1[1, i], display_mode='z', colorbar=False, bg_img=bg_img,title='{} Task : Low Pleasance (fdr=0.05, threshold = {})'.format(task,threshold))
        roi_img.add_markers([coords[i]], 'green', 300)
    fig1.set_size_inches(50, 19)
    plt.show()


    #PLOT 2
    fig2, axs2 = plt.subplots(2, 3)
    for i in range(3, 6):
        axs2[0, i-3].set_title(' Top {} Activated Cluster \n(cluster peak voxel at coord {})'.format(i+1,coords[i]))
        axs2[0, i-3].plot(real_timeseries[:, i], c='blue', lw=2,label = 'Actual Bold Signal')
        axs2[0, i-3].plot(predicted_timeseries[:, i], c='r',  ls='-', lw=2,label= 'Predicted Bold Signal')
        axs2[0, i-3].set_xlabel('Time')
        axs2[0, i-3].set_ylabel('Signal intensity', labelpad=0)
        axs2[0, i-3].legend()
        roi_img = plotting.plot_stat_map(
            z_map, cut_coords=[coords[i][2]], threshold=threshold, figure=fig2,
            axes=axs2[1, i-3], display_mode='z', colorbar=False, bg_img=bg_img,title='{} Task : Low Pleasance (fdr=0.05, threshold = {})'.format(task,threshold))
        roi_img.add_markers([coords[i]], 'green', 300)
    fig2.set_size_inches(50, 19)
    plt.show()

        

In [5]:
def stat_result_long(z_map,contrast_name,task,alpha,method,bg_img):
    
    _, threshold = map_threshold(z_map, alpha=alpha, height_control=method)
    table = get_clusters_table(z_map, stat_threshold=threshold)
    plot_stat_map(z_map, bg_img=bg_img, threshold=threshold,
                  display_mode = 'z', cut_coords=5,
                   black_bg=True,
                  title='{} Task : {} ({}={}, threshold = {})'.format(task,contrast_name,method,alpha,threshold))
    plt.show()
    return table, threshold

In [6]:
def stat_result(z_map,contrast_name,task,alpha,method,bg_img):
    
    _, threshold = map_threshold(z_map, alpha=alpha, height_control=method)
    table = get_clusters_table(z_map, stat_threshold=threshold)
    plot_stat_map(z_map, bg_img=bg_img, threshold=threshold,
                   black_bg=True,
                  title='{} Task : {} ({}={}, threshold = {})'.format(task,contrast_name,method,alpha,threshold))
    plt.show()
    return table, threshold

## MVPA

In [7]:
def lss_separate_method_comp(task,
                          data_path,
                          events_path,
                          motion_param_path,
                          cache_mem_path,
                          activity_mat_saving_path,
                          analysis_type,
                          param_modulation,
                          t_r,
                          drift_model,
                          drift_order,
                          hrf_model,
                          pre_whitening_spec,
                          apply_trans_mask,
                          roi_img,  
                          hpf_cut,
                          smoothing_fwhm,
                          standardize,
                          detrend,minimize_memory):


    output_activity_matrices = dict()                          
    # Data processing                      

    data = image.load_img(data_path)
    m_img = mean_img(data) # mean niimg for plotting purposes
    motion = pd.read_csv(motion_param_path,sep=',',header=None)
    motion = motion[list(range(1,7))]
    motion_col = ['tx', 'ty', 'tz', 'rx', 'ry', 'rz']
    motion.columns = motion_col


    #Experimental Paradigm Processing

    events = pd.read_csv(events_path,sep='\t')
    events.columns = ['onset','duration','trial_type','modulation']
    events.dropna(inplace=True)
    events.index = range(len(events))



    if analysis_type == 'High_Low_Pleasance':

        events['trial_type_desc'] = ['High Pleasance' if x >=50 else 'Low Pleasance' for x in events['modulation'] ]

        if param_modulation:

            events = events[['onset','duration','trial_type_desc','modulation']]
            events.columns = ['onset','duration','trial_type','modulation']
        else:
            events = events[['onset','duration','trial_type_desc']]
            events.columns = ['onset','duration','trial_type']



    elif analysis_type == 'High_Mid_Low_Pleasance':

        events['trial_type_desc'] = ['High Pleasance' if x >=67 else 'Mid Pleasance' if x>=34 else 'Low Pleasance' for x in events['modulation'] ]

        if param_modulation:

            events = events[['onset','duration','trial_type_desc','modulation']]
            events.columns = ['onset','duration','trial_type','modulation']
        else:
            events = events[['onset','duration','trial_type_desc']]
            events.columns = ['onset','duration','trial_type']



    #  LSS - Separate event matrices computation


    mvpa_events_labels= list()
    for i in range(len(events['trial_type'])):

        mvpa_event_label = events['trial_type'][i]

        vars()['events_{}_{}'.format(mvpa_event_label.replace(' ',''),i+1)] = events.copy()

        mvpa_events_labels.append('{}_{}'.format(mvpa_event_label.replace(' ',''),i+1))

        trial_type_treated = ['{}_{}'.format(mvpa_event_label,i+1) if k == i else 'Nuisance Event' for k in  range(len(events['trial_type']))]

        vars()['events_{}_{}'.format(mvpa_event_label.replace(' ',''),i+1)]['trial_type'] = trial_type_treated



        print('Event {}'.format(i))
        print(vars()['events_{}_{}'.format(mvpa_event_label.replace(' ',''),i+1)])

    # Multiple GLM fit


    for mvpa_label in mvpa_events_labels:
        print('GLM RUN {}'.format(mvpa_label))

        # Design Matrix Computation + plotting

        frametimes = np.linspace(0, (data.shape[3] - 1) * t_r, data.shape[3])




        design_matrix = make_first_level_design_matrix(frametimes,
                           events=vars()['events_{}'.format(mvpa_label)],
                           add_regs=motion,
                           add_reg_names=motion_col,
                           hrf_model= hrf_model,
                           fir_delays = None,
                           drift_model= drift_model,
                           drift_order =drift_order)

        plot_design_matrix(design_matrix)

        # Statistically Enhancing Mask Computation

        if apply_trans_mask :


            masker = input_data.NiftiMasker(mask_img=roi_img,smoothing_fwhm=smoothing_fwhm,
                                standardize=standardize,detrend=detrend,
                                high_pass=hpf_cut,t_r=t_r).fit(data)

        else :

            masker = input_data.NiftiMasker(mask_img=roi_img).fit(data)


        # GLM instantiation and fit

        glm = FirstLevelModel(t_r=t_r,
                          mask_img=masker,
                          noise_model= pre_whitening_spec,
                          high_pass=hpf_cut,
                          smoothing_fwhm = smoothing_fwhm,
                          standardize=standardize,
                          memory=cache_mem_path,
                          minimize_memory=minimize_memory, 
                          verbose=1)

        glm = glm.fit(data,design_matrices=design_matrix)



        design_matrix = glm.design_matrices_[0]
        plot_design_matrix(design_matrix)
        plt.title('Design Matrix')
        plt.show()

        masker = NiftiMasker()

        vars()['{}_activity_matrix'.format(mvpa_label)] = pd.DataFrame(None)


        condition = np.eye(N=1,M=len(design_matrix.columns),k=0)
        plot_contrast_matrix(condition,design_matrix=design_matrix)
        z_map = glm.compute_contrast(condition,
                                  stat_type='t',
                                  output_type='z_score')
        masked_map = masker.fit_transform(z_map)

        vars()['{}_activity_matrix'.format(mvpa_label)] = vars()['{}_activity_matrix'.format(mvpa_label)].append(pd.DataFrame(masked_map))

        print(vars()['{}_activity_matrix'.format(mvpa_label)])

        print('ACTIVITY MATRIX {}'.format(mvpa_label))

        if activity_mat_saving_path != None:

            vars()['{}_activity_matrix'.format(mvpa_label)].to_csv('{}{}_activity_matrix.csv'.format(activity_mat_saving_path,mvpa_label)) 



    # Control Standardization



    if analysis_type == 'High_Low_Pleasance':

        HP_activity_matrix = list()
        LP_activity_matrix = list()


        for mvpa_label in mvpa_events_labels:

            if mvpa_label[0] == 'H':
                HP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])
            else:
                LP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])

        for label in ['HP','LP'] :

            activity_matrix = np.stack(vars()['{}_activity_matrix'.format(label)],axis=2)



            activity_matrix_dim_voxel_mean = activity_matrix.mean(axis=2)
            activity_matrix_dim_voxel_std = activity_matrix.std(axis=2)


            activity_matrix_dim_voxel_mean.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)
            activity_matrix_dim_voxel_std.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)


            activity_matrix_c_standardized = (activity_matrix - activity_matrix_dim_voxel_mean)/activity_matrix_dim_voxel_std

            output_activity_matrices[label] = activity_matrix_c_standardized




    elif analysis_type == 'High_Mid_Low_Pleasance':



        HP_activity_matrix = list()
        MP_activity_matrix = list() 
        LP_activity_matrix = list()

        for mvpa_label in mvpa_events_labels:
            if mvpa_label[0] == 'H':
                HP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])
            elif mvpa_label[0] == 'M' :
                MP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])
            else:
                LP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)]) 


        for label in ['HP','LP'] :

            activity_matrix = np.stack(vars()['{}_activity_matrix'.format(label)],axis=2)



            activity_matrix_dim_voxel_mean = activity_matrix.mean(axis=2)
            activity_matrix_dim_voxel_std = activity_matrix.std(axis=2)


            activity_matrix_dim_voxel_mean.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)
            activity_matrix_dim_voxel_std.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)


            activity_matrix_c_standardized = (activity_matrix - activity_matrix_dim_voxel_mean)/activity_matrix_dim_voxel_std

            output_activity_matrices[label] = activity_matrix_c_standardized


    # Feature Matrix and Label Vector Computation    

    label_vect = list()

    for label in output_activity_matrices.keys():
        vec = np.array([label for i in range(output_activity_matrices[label].shape[2])])
        vec.shape = (output_activity_matrices[label].shape[2],1)
        label_vect.append(vec)

    y = np.vstack(label_vect)

    features_mat = list()

    for label in output_activity_matrices.keys():
        mat = output_activity_matrices[label].copy()
        features_mat.append(mat.reshape(mat.shape[0]*mat.shape[1],mat.shape[2]).T)



        X = np.vstack(features_mat)


    return  y, X

In [8]:
def fbr_separate_method_comp(task,
                          data_path,
                          events_path,
                          motion_param_path,
                          cache_mem_path,
                          activity_mat_saving_path,
                          analysis_type,
                          param_modulation,
                          t_r,
                          drift_model,
                          drift_order,
                          fbr_expected_tr_delay,
                          pre_whitening_spec,
                          apply_trans_mask,
                          roi_img,  
                          hpf_cut,
                          smoothing_fwhm,
                          standardize,
                          detrend,minimize_memory):
    
    
    output_activity_matrices = dict()                          
    # Data processing                      

    data = image.load_img(data_path)
    m_img = mean_img(data) # mean niimg for plotting purposes
    motion = pd.read_csv(motion_param_path,sep=',',header=None)
    motion = motion[list(range(1,7))]
    motion_col = ['tx', 'ty', 'tz', 'rx', 'ry', 'rz']
    motion.columns = motion_col


    #Experimental Paradigm Processing

    events = pd.read_csv(events_path,sep='\t')
    events.columns = ['onset','duration','trial_type','modulation']
    events.dropna(inplace=True)
    events.index = range(len(events))
    
    
    
    if analysis_type == 'High_Low_Pleasance':
        
        events['trial_type_desc'] = ['High Pleasance' if x >=50 else 'Low Pleasance' for x in events['modulation'] ]
        
        if param_modulation:
           
            events = events[['onset','duration','trial_type_desc','modulation']]
            events.columns = ['onset','duration','trial_type','modulation']
        else:
            events = events[['onset','duration','trial_type_desc']]
            events.columns = ['onset','duration','trial_type']
            


    elif analysis_type == 'High_Mid_Low_Pleasance':
        
        events['trial_type_desc'] = ['High Pleasance' if x >=67 else 'Mid Pleasance' if x>=34 else 'Low Pleasance' for x in events['modulation'] ]
        
        if param_modulation:
           
            events = events[['onset','duration','trial_type_desc','modulation']]
            events.columns = ['onset','duration','trial_type','modulation']
        else:
            events = events[['onset','duration','trial_type_desc']]
            events.columns = ['onset','duration','trial_type']
            
            
            
    #  FBR - Separate event matrices computation

    
    mvpa_events_labels= list()
    for i in range(len(events['trial_type'])):
    
        mvpa_event_label = events['trial_type'][i]

        vars()['events_{}_{}'.format(mvpa_event_label.replace(' ',''),i+1)] = events.copy()

        mvpa_events_labels.append('{}_{}'.format(mvpa_event_label.replace(' ',''),i+1))

        trial_type_treated = ['{}_{}'.format(mvpa_event_label,i+1) if k == i else 'Nuisance Event' for k in  range(len(events['trial_type']))]

        vars()['events_{}_{}'.format(mvpa_event_label.replace(' ',''),i+1)]['trial_type'] = trial_type_treated



        print('Event {}'.format(i))
        print(vars()['events_{}_{}'.format(mvpa_event_label.replace(' ',''),i+1)])
            
    # Multiple GLM fit
    
    
    for mvpa_label in mvpa_events_labels:
        print('GLM RUN {}'.format(mvpa_label))
        
        # Design Matrix Computation + plotting
        
        frametimes = np.linspace(0, (data.shape[3] - 1) * t_r, data.shape[3])
    
        fir_delays = list(range(1,fbr_expected_tr_delay+1)) if fbr_expected_tr_delay != None else None
    

        design_matrix = make_first_level_design_matrix(frametimes,
                           events=vars()['events_{}'.format(mvpa_label)],
                           add_regs=motion,
                           add_reg_names=motion_col,
                           hrf_model='fir',
                           fir_delays = fir_delays,
                           drift_model= drift_model,
                           drift_order =drift_order)

        plot_design_matrix(design_matrix)

        # Statistically Enhancing Mask Computation

        if apply_trans_mask :
    
    
            masker = input_data.NiftiMasker(mask_img=roi_img,smoothing_fwhm=smoothing_fwhm,
                                standardize=standardize,detrend=detrend,
                                high_pass=hpf_cut,t_r=t_r).fit(data)
      
        else :
        
            masker = input_data.NiftiMasker(mask_img=roi_img).fit(data)


        # GLM instantiation and fit

        glm = FirstLevelModel(t_r=t_r,
                          mask_img=masker,
                          noise_model= pre_whitening_spec,
                          high_pass=hpf_cut,
                          smoothing_fwhm = smoothing_fwhm,
                          standardize=standardize,
                          memory=cache_mem_path,
                          minimize_memory=minimize_memory, 
                          verbose=1)

        glm = glm.fit(data,design_matrices=design_matrix)
        
    
    
        design_matrix = glm.design_matrices_[0]
        plot_design_matrix(design_matrix)
        plt.title('Design Matrix')
        plt.show()

        masker = NiftiMasker()

        vars()['{}_activity_matrix'.format(mvpa_label)] = pd.DataFrame(None)

        for i in range(fbr_expected_tr_delay):
            condition_delay = np.eye(N=1,M=len(design_matrix.columns),k=i)
            plot_contrast_matrix(condition_delay,design_matrix=design_matrix)
            z_map = glm.compute_contrast(condition_delay,
                                      stat_type='t',
                                      output_type='z_score')
            masked_map = masker.fit_transform(z_map)

            vars()['{}_activity_matrix'.format(mvpa_label)] = vars()['{}_activity_matrix'.format(mvpa_label)].append(pd.DataFrame(masked_map))



        print('ACTIVITY MATRIX {}'.format(mvpa_label))
                                         
        if activity_mat_saving_path != None:
                                         
            vars()['{}_activity_matrix'.format(mvpa_label)].to_csv('{}{}_activity_matrix.csv'.format(activity_mat_saving_path,mvpa_label)) 
        
                                         
                                         
    # Control Standardization
                                         
                                         
                                         
    if analysis_type == 'High_Low_Pleasance':

        HP_activity_matrix = list()
        LP_activity_matrix = list()
        
                                         
        for mvpa_label in mvpa_events_labels:
                                         
            if mvpa_label[0] == 'H':
                HP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])
            else:
                LP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])
         
        for label in ['HP','LP'] :
             
            activity_matrix = np.stack(vars()['{}_activity_matrix'.format(label)],axis=2)
             
                                         
                                         
            activity_matrix_dim_voxel_mean = activity_matrix.mean(axis=2)
            activity_matrix_dim_voxel_std = activity_matrix.std(axis=2)


            activity_matrix_dim_voxel_mean.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)
            activity_matrix_dim_voxel_std.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)


            activity_matrix_c_standardized = (activity_matrix - activity_matrix_dim_voxel_mean)/activity_matrix_dim_voxel_std
                                         
            output_activity_matrices[label] = activity_matrix_c_standardized
                                         
            
            
                                         
    elif analysis_type == 'High_Mid_Low_Pleasance':
                                         
                                         
        
        HP_activity_matrix = list()
        MP_activity_matrix = list() 
        LP_activity_matrix = list()
        
        for mvpa_label in mvpa_events_labels:
            if mvpa_label[0] == 'H':
                HP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])
            elif mvpa_label[0] == 'M' :
                MP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)])
            else:
                LP_activity_matrix.append(vars()['{}_activity_matrix'.format(mvpa_label)]) 
                                         
         
        for label in ['HP','LP'] :
             
            activity_matrix = np.stack(vars()['{}_activity_matrix'.format(label)],axis=2)
             
                                         
                                         
            activity_matrix_dim_voxel_mean = activity_matrix.mean(axis=2)
            activity_matrix_dim_voxel_std = activity_matrix.std(axis=2)


            activity_matrix_dim_voxel_mean.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)
            activity_matrix_dim_voxel_std.shape = (activity_matrix.shape[0],activity_matrix.shape[1],1)


            activity_matrix_c_standardized = (activity_matrix - activity_matrix_dim_voxel_mean)/activity_matrix_dim_voxel_std
                                         
            output_activity_matrices[label] = activity_matrix_c_standardized
                                         
                
    # Feature Matrix and Label Vector Computation    
                
    label_vect = list()
    
    for label in output_activity_matrices.keys():
        vec = np.array([label for i in range(output_activity_matrices[label].shape[2])])
        vec.shape = (output_activity_matrices[label].shape[2],1)
        label_vect.append(vec)
        
    y = np.vstack(label_vect)
    
    features_mat = list()

    for label in output_activity_matrices.keys():
        mat = output_activity_matrices[label].copy()
        features_mat.append(mat.reshape(mat.shape[0]*mat.shape[1],mat.shape[2]).T)



    X = np.vstack(features_mat)
        
    
    return  y, X

# I/O

####  Google Drive Path Initialization

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
cd drive/MyDrive/CEA/

/content/drive/MyDrive/CEA


In [11]:
ls

cache_mem/  data/  mask/  models/  mvpa/  ses-19/


## Global  Statistical Computation Parameters

In [12]:
voxel_dim = 1.5
t_r = 2
pre_whitening_spec = 'ar1'
smoothing_fwhm = 2*voxel_dim 
slice_time_ref = 0. # Matching with STC done in preproc
cache_mem_path = None

# Multi Voxel Pattern Analysis

### To update

![image.png](attachment:image.png)

# ROI SELECTION

### Based on the global preference FFX ( across tasks and runs)

#### High Pleasance / Low Pleasance Classification

In [13]:
mask_img = image.load_img('mask/linear_vmpfc_mask.nii.gz')

## Activity Matrices Extraction ( LSS Separate Method )

 - We extract the activity matrices/ label vector pair for each run, for each task
 - We perform an onset shift to capture the very moment of the decision, we set it to 1000 ms before the button press
 - We concatenate all this data to create a global dataset over all the preference battery of test

In [14]:
y_g = list()
X_g = list()

### Faces Task

### Run AP

In [15]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferenceFaces_dir-ap_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferenceFaces_dir-ap_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferenceFaces_dir-ap_bold.txt'
task = 'Faces'

In [16]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1


onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)

hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [17]:
y_fa_ap, X_fa_ap = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [18]:
y_g.append(y_fa_ap)
X_g.append(X_fa_ap)

In [19]:
X_fa_ap.shape

(60, 3)

### Run PA

In [20]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferenceFaces_dir-pa_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferenceFaces_dir-pa_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferenceFaces_dir-pa_bold.txt'
task = 'Faces'

In [21]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1

onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)
hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [22]:
y_fa_pa, X_fa_pa = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
y_g.append(y_fa_pa)
X_g.append(X_fa_pa)

In [24]:
X_fa_pa.shape

(60, 3)

### Food Task

### Run AP

In [25]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferenceFood_dir-ap_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferenceFood_dir-ap_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferenceFood_dir-ap_bold.txt'
task = 'Food'

In [26]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1

onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)
hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [27]:
y_fo_ap, X_fo_ap = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [28]:
y_g.append(y_fo_ap)
X_g.append(X_fo_ap)

In [29]:
X_fo_ap.shape

(60, 3)

### Run PA

In [30]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferenceFood_dir-pa_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferenceFood_dir-pa_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferenceFood_dir-pa_bold.txt'
task = 'Food'

In [31]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1

onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)
hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [32]:
y_fo_pa, X_fo_pa = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [33]:
y_g.append(y_fo_pa)
X_g.append(X_fo_pa)

In [34]:
X_fo_pa.shape

(60, 3)

### Houses Task

### Run AP

In [35]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferenceHouses_dir-ap_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferenceHouses_dir-ap_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferenceHouses_dir-ap_bold.txt'
task = 'Houses'

In [36]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1

onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)
hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [37]:
y_ho_ap, X_ho_ap = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [38]:
y_g.append(y_ho_ap)
X_g.append(X_ho_ap)

In [39]:
X_ho_ap.shape

(60, 3)

### Run PA

In [40]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferenceHouses_dir-pa_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferenceHouses_dir-pa_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferenceHouses_dir-pa_bold.txt'
task = 'Houses'

In [41]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1

onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)
hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [42]:
y_ho_pa, X_ho_pa = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [43]:
y_g.append(y_ho_pa)
X_g.append(X_ho_pa)

In [44]:
X_ho_pa.shape

(60, 3)

### Paintings Task

### Run AP

In [45]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferencePaintings_dir-ap_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferencePaintings_dir-ap_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferencePaintings_dir-ap_bold.txt'
task = 'Paintings'

In [46]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1

onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)
hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [47]:
y_p_ap, X_p_ap = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [48]:
y_g.append(y_p_ap)
X_g.append(X_p_ap)

### Run PA

In [49]:
data_path = 'ses-19/func/wrdcsub-01_ses-19_task-PreferencePaintings_dir-pa_bold.nii.gz'
events_path = 'ses-19/func/sub-01_ses-19_task-PreferencePaintings_dir-pa_events.tsv'
motion_param_path = 'ses-19/func/rp_dcsub-01_ses-19_task-PreferencePaintings_dir-pa_bold.txt'
task = 'Paintings'

In [50]:
analysis_type = 'High_Low_Pleasance'
hrf_model = 'spm + derivative'
param_modulation = False
drift_model='cosine'
t_r=2
drift_order=None
apply_trans_mask=True
roi_img = mask_img
standardize=True
detrend=True
minimize_memory = True
activity_mat_saving_path = 'mvpa/test2'

events = pd.read_csv(events_path,sep='\t')

# onset shift in order to capture the very moment of the appreciation decision
events['onset'] = events['onset']-1

onset_periods = [events['onset'][i+1] - events['onset'][i]  for i in range(len(events['onset']))if (i<len(events['onset'])-1)]
onset_periods = np.array(onset_periods)
onset_freq = np.reciprocal(onset_periods)
hpf_cut = np.min(onset_freq)/2 # cf High Pass Filtering Rule of Thumb ( fMRI Handbook 2011)

In [51]:
y_p_pa, X_p_pa = lss_separate_method_comp(task =task,
                          data_path=data_path,
                          events_path=events_path,
                          motion_param_path=motion_param_path,
                          cache_mem_path=cache_mem_path,
                          activity_mat_saving_path=activity_mat_saving_path,
                          analysis_type=analysis_type,
                          param_modulation=False,
                          t_r=t_r,
                          drift_model=drift_model,
                          drift_order=drift_order,
                          hrf_model=hrf_model,
                          pre_whitening_spec=pre_whitening_spec,
                          apply_trans_mask=apply_trans_mask,
                          roi_img=roi_img,  
                          hpf_cut=hpf_cut,
                          smoothing_fwhm=smoothing_fwhm,
                          standardize=standardize,
                          detrend=detrend,minimize_memory=minimize_memory)

Output hidden; open in https://colab.research.google.com to view.

In [52]:
y_g.append(y_p_pa)
X_g.append(X_p_pa)

### Global Dataset Construction

In [53]:
y = np.vstack(y_g)
X = np.vstack(X_g)

In [54]:
X.shape

(480, 3)

In [55]:
X

array([[ 2.33376215,  2.17963908,  1.48816649],
       [-0.44597196,  0.07732953,  0.40238784],
       [-0.01551307,  0.18509338,  0.60893397],
       ...,
       [-0.97798119, -0.67005892, -1.2307879 ],
       [ 0.26154514,  0.29808347,  0.39118863],
       [-2.14594286, -2.19705447, -2.25353248]])

In [56]:
y.shape

(480, 1)

In [57]:
y

array([['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['LP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['HP'],
       ['H

In [58]:
np.save('mvpa/X_pref_lss_shift_linear_vmpfc',X)
np.save('mvpa/y_pref_lss_shift_linear_vmpfc',y)
